In [7]:
import numpy as np
import scipy.io as sio
import cv2
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier
import optuna

# Load data
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\new.mat'
new = sio.loadmat(file_path)
ET = new['ET']
Label_array_ET = new['label_list'].ravel()
EEG = new['EEG']

# Reshape data for SMOTE
raw_data = ET.reshape(ET.shape[0] * ET.shape[1], -1).T

# Apply SMOTE on the raw data
smote = SMOTE(random_state=42)
raw_data_resampled, Label_array_ET_resampled = smote.fit_resample(raw_data, Label_array_ET)

# Reshape resampled data back to the original shape
ET_resampled = raw_data_resampled.T.reshape(ET.shape[0], ET.shape[1], -1)

# Create an empty list to store the images
image_list = []

# Handcrafted features
handcrafted_features = []

# Loop through each sample
for sample_idx in range(ET_resampled.shape[2]):
    # Extract X and Y coordinates for both eyes
    x_left, y_left, pupil_left, x_right, y_right, pupil_right = ET_resampled[:, :, sample_idx]
    
    # Create a blank canvas (64x64) for the gaze plot
    gaze_plot = np.zeros((64, 64, 3), dtype=np.uint8)  # Initialize as black image

    # Scale the coordinates to match the screen size
    scaled_x_left = (x_left * 1920 / 30).astype(int)  # 1920 / 30 ~ 64
    scaled_y_left = (y_left * 1080 / 16.875).astype(int)  # 1080 / 16.875 ~ 64
    scaled_x_right = (x_right * 1920 / 30).astype(int)
    scaled_y_right = (y_right * 1080 / 16.875).astype(int)

    # Set gaze points as white pixels on the canvas
    for i in range(120):
        if 0 <= scaled_x_left[i] < 64 and 0 <= scaled_y_left[i] < 64:
            gaze_plot[scaled_y_left[i], scaled_x_left[i]] = [255, 255, 255]  # White color
        if 0 <= scaled_x_right[i] < 64 and 0 <= scaled_y_right[i] < 64:
            gaze_plot[scaled_y_right[i], scaled_x_right[i]] = [255, 255, 255]  # White color

    # Append the gaze plot to the image list
    image_list.append(gaze_plot)

    # Extract handcrafted features
    mean_left = np.mean([x_left, y_left, pupil_left])
    std_left = np.std([x_left, y_left, pupil_left])
    mean_right = np.mean([x_right, y_right, pupil_right])
    std_right = np.std([x_right, y_right, pupil_right])
    
    # Append handcrafted features
    handcrafted_features.append([mean_left, std_left, mean_right, std_right])

# Convert the list of images to a NumPy array
image_array = np.array(image_list)
handcrafted_features = np.array(handcrafted_features)

# Reshape images to have a single channel (grayscale)
image_array = np.expand_dims(image_array, axis=-1)

# Normalize the image data to the range [0, 1]
image_array = image_array / 255.0

# Split the data into training and test sets
x_train_img, x_test_img, x_train_hand, x_test_hand, y_train, y_test = train_test_split(
    image_array, handcrafted_features, Label_array_ET_resampled, test_size=0.3, random_state=42)

# Define LeNet-5 for feature extraction
lenet5 = models.Sequential([
    layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten()
])

# Extract features using LeNet-5
lenet5_features_train = lenet5.predict(x_train_img)
lenet5_features_test = lenet5.predict(x_test_img)

# Combine handcrafted features with LeNet-5 features
x_train_combined = np.concatenate((lenet5_features_train, x_train_hand), axis=1)
x_test_combined = np.concatenate((lenet5_features_test, x_test_hand), axis=1)

# Function for Optuna optimization
def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    
    # Define the ensemble classifier
    estimators = [
        ('rf', RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, random_state=42)),
        ('xgb', XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, subsample=subsample, colsample_bytree=colsample_bytree, use_label_encoder=False, eval_metric='logloss', random_state=42))
    ]
    
    ensemble_clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(n_estimators=100, random_state=42))
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(ensemble_clf, x_train_combined, y_train, cv=skf, scoring='roc_auc')
    
    return np.mean(scores)

# Optimize hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

# Get the best hyperparameters
best_params = study.best_params

# Define the ensemble classifier with best hyperparameters
estimators = [
    ('rf', RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], subsample=best_params['subsample'], random_state=42)),
    ('xgb', XGBClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], learning_rate=best_params['learning_rate'], subsample=best_params['subsample'], colsample_bytree=best_params['colsample_bytree'], use_label_encoder=False, eval_metric='logloss', random_state=42))
]

ensemble_clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(n_estimators=100, random_state=42))

# Train the ensemble classifier
ensemble_clf.fit(x_train_combined, y_train)

# Evaluate the ensemble classifier
y_pred = ensemble_clf.predict(x_test_combined)
y_pred_prob = ensemble_clf.predict_proba(x_test_combined)[:, 1]

# Calculate and print the AUC
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC: {auc:.4f}")

# Plot AUC-ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Print classification report
print(classification_report(y_test, y_pred))


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


[I 2024-05-27 01:58:26,942] A new study created in memory with name: no-name-5b4aada8-ac76-428a-8ae0-dab13cfe5238
[I 2024-05-27 03:53:59,313] Trial 0 finished with value: 0.5908266466196238 and parameters: {'n_estimators': 59, 'max_depth': 5, 'learning_rate': 0.0011490394246152652, 'subsample': 0.6926982467930112, 'colsample_bytree': 0.5074292779558145}. Best is trial 0 with value: 0.5908266466196238.
[I 2024-05-27 16:24:22,039] Trial 1 finished with value: 0.6322152827803349 and parameters: {'n_estimators': 78, 'max_depth': 8, 'learning_rate': 0.002500250850244193, 'subsample': 0.9167852584671607, 'colsample_bytree': 0.6414209231291539}. Best is trial 1 with value: 0.6322152827803349.
[W 2024-05-27 16:53:48,618] Trial 2 failed with parameters: {'n_estimators': 77, 'max_depth': 16, 'learning_rate': 0.00016593943473621855, 'subsample': 0.9404422683493163, 'colsample_bytree': 0.8340373187608296} because of the following error: ValueError('\nAll the 10 fits failed.\nIt is very likely that

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_stacking.py", line 669, in fit
    return super().fit(X, y_encoded, sample_weight)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\UC\AppData\Roaming

In [2]:
from sklearn.metrics import classification_report

# Predict on the test set
y_pred = ensemble_clf.predict(x_test_combined)

# Print classification report
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])
print(report)


              precision    recall  f1-score   support

     Class 0       0.70      0.73      0.71      2100
     Class 1       0.70      0.67      0.69      2024

    accuracy                           0.70      4124
   macro avg       0.70      0.70      0.70      4124
weighted avg       0.70      0.70      0.70      4124

